This notebook is only for COLAB.

In [ ]:
!wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/ocr/requirements.txt
!pip install -r requirements.txt
!wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/ocr/app.py

!mkdir examples
!cd examples && wget https://github.com/mrsyee/dl_apps/main/ocr/examples/Red.png
!cd examples && wget https://github.com/mrsyee/dl_apps/main/ocr/examples/sentence.png

In [ ]:
from app import app
app.launch()